In [3]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="Spark1")

In [7]:
full_text = sc.textFile("./pg100.txt")
cleaner = lambda word: False if (word.isupper() or word.isdigit() or (word[:-1].isupper() and word[-1] == '.')) else True
full_text = full_text.flatMap(lambda line: line.split(" ")).filter(cleaner)
full_text.take(5)
full_text.map(lambda x)

[u'The', u'Project', u'Gutenberg', u'EBook', u'of']